In [ ]:
import os
import h5py
import time
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import json
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
seed = 1747265027
# seed = int(time.time()) % (2**32 - 1)  # ou: random.randint(0, 999999)
print(f"Usando seed: {seed}")

os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

In [ ]:
train_path, valid_path, test_path = 'train.h5', 'valid.h5', 'test.h5'
processed_train, processed_valid, processed_test = '_train.h5', '_valid.h5', '_test.h5'
channels = [0, 3]
img_w = 64
load_batch = 4096
epochs = 500
batch = 8

In [ ]:
def cut_images(images, width):
    start = images.shape[1] // 2 - width // 2
    end = images.shape[1] // 2 + width // 2
    return images[:, start:end, start:end, :]

In [ ]:
def clean_images(images):
    images = np.nan_to_num(images, copy=False)
    images[images > 1000] = 0
    return images

In [ ]:
def split_data(path, train_dest, valid_dest, test_dest, channels=[0, 1, 2, 3], batch=1024):

    # abre o arquivo com os dados e os arquivos de destino
    with h5py.File(path, 'r') as srcf, \
            h5py.File(train_dest, 'w') as train_w, \
            h5py.File(valid_dest, 'w') as valid_w, \
            h5py.File(test_dest, 'w') as test_w:
        
        images = srcf['matrix']
        info = pd.read_hdf(path, key='info', mode='r')

        # recupera o tamanho dos dados
        data_len = info.shape[0]

        # itera sobre todos os dados em chuncks
        for chunck in range(0, data_len, batch):
            # seleciona o chunck
            chunck_slc = slice(chunck, chunck + batch if chunck + batch < data_len else data_len)

            # lê os dados do chunck
            img_chunck = images[chunck_slc]
            info_chunck = info[chunck_slc]
            info_chunck = info_chunck['Vmax']

            # seleciona os canais
            img_chunck = img_chunck[:, :, :, channels]
            
            img_new_shape = img_chunck.shape[1:]

            # separa os dados
            train_img, test_img, train_info, test_info = train_test_split(img_chunck, info_chunck, test_size=.3, random_state=seed)
            test_img, valid_img, test_info, valid_info = train_test_split(test_img, test_info, test_size=.5, random_state=seed)

            # escreve no arquivo de treinamento
            if 'matrix' not in train_w:
                train_w.create_dataset('matrix', shape=(0,) + img_new_shape, maxshape=(None,) + img_new_shape)
            train_w['matrix'].resize(train_w['matrix'].shape[0] + train_img.shape[0], axis=0)
            train_w['matrix'][-train_img.shape[0]:] = train_img
            if 'info' not in train_w:
                train_w.create_dataset('info', shape=(0,), maxshape=(None,))
            train_w['info'].resize(train_w['info'].shape[0] + train_info.shape[0], axis=0)
            train_w['info'][-train_info.shape[0]:] = train_info

            # escreve no arquivo de validação
            if 'matrix' not in valid_w:
                valid_w.create_dataset('matrix', shape=(0,) + img_new_shape, maxshape=(None,) + img_new_shape)
            valid_w['matrix'].resize(valid_w['matrix'].shape[0] + valid_img.shape[0], axis=0)
            valid_w['matrix'][-valid_img.shape[0]:] = valid_img
            if 'info' not in valid_w:
                valid_w.create_dataset('info', shape=(0,), maxshape=(None,))
            valid_w['info'].resize(valid_w['info'].shape[0] + valid_info.shape[0], axis=0)
            valid_w['info'][-valid_info.shape[0]:] = valid_info

            # escreve no arquivo de teste
            if 'matrix' not in test_w:
                test_w.create_dataset('matrix', shape=(0,) + img_new_shape, maxshape=(None,) + img_new_shape)
            test_w['matrix'].resize(test_w['matrix'].shape[0] + test_img.shape[0], axis=0)
            test_w['matrix'][-test_img.shape[0]:] = test_img
            if 'info' not in test_w:
                test_w.create_dataset('info', shape=(0,), maxshape=(None,))
            test_w['info'].resize(test_w['info'].shape[0] + test_info.shape[0], axis=0)
            test_w['info'][-test_info.shape[0]:] = test_info

ds = 'TCIR-ATLN_EPAC_WPAC.h5'
split_data(f'../data/{ds}', train_path, valid_path, test_path, channels=channels, batch=load_batch)

In [ ]:
with h5py.File(train_path, mode='r') as trainsrc, h5py.File(valid_path, mode='r') as validsrc :
    data_len = trainsrc['matrix'].shape[0]
    valid_data_len = validsrc['matrix'].shape[0]
    print('Dataset de treino: ', trainsrc['matrix'].shape)
    print('Dataset de validação: ', validsrc['matrix'].shape)

iter_train = data_len // batch
iter_valid = valid_data_len // batch

In [ ]:
def get_mean(file, batch=1024, width=64):
    with h5py.File(file, mode='r') as src:
        images = src['matrix']
        data_len = images.shape[0]
        accumulators = np.zeros(images.shape[-1])
        for i in range(0, data_len, batch):
            start = images.shape[1] // 2 - width // 2
            end = images.shape[1] // 2 + width // 2
            slc = slice(start, end)
            chunck = images[i: i + batch if i + batch < data_len else data_len, slc, slc]
            chunck = clean_images(chunck)
            for j in range(accumulators.shape[0]):
                accumulators[j] += np.sum(chunck[:, :, :, j])
    means = accumulators / (data_len * width * width)
    return means

mean = get_mean(train_path, batch=load_batch, width=img_w)
print(mean)

In [ ]:
def get_std(file, mean, batch=1024, width=64):
    with h5py.File(file, mode='r') as src:
        images = src['matrix']
        data_len = images.shape[0]
        accumulators = np.zeros(images.shape[-1])
        for i in range(0, data_len, batch):
            start = images.shape[1] // 2 - width // 2
            end = images.shape[1] // 2 + width // 2
            slc = slice(start, end)
            chunck = images[i: i + batch if i + batch < data_len else data_len, slc, slc]
            chunck = clean_images(chunck)
            for j in range(accumulators.shape[0]):
                accumulators[j] += np.sum((chunck[:, :, :, j] - mean[j]) ** 2)
    stds = accumulators / (data_len * width * width)
    stds = np.sqrt(stds)
    return stds

std = get_std(train_path, mean, batch=load_batch, width=img_w)
print(std)

In [ ]:

def pre_process(file, dest, width, means, stds, batch=1024):
    # corta imagem grande o suficiente para poder rotacionar
    rotation_width = int(np.ceil(np.sqrt((width ** 2) * 2)))
    if rotation_width % 2 != 0:
        rotation_width += 1
    with h5py.File(file, mode='r') as src, h5py.File(dest, mode='w') as destf:
        src_imgs = src['matrix']
        data_len = src_imgs.shape[0]
        for i in range(0, data_len, batch):
            slc = slice(i, i + batch if i + batch < data_len else data_len)
            img_chunck = src_imgs[slc]
            
            img_chunck = cut_images(img_chunck, rotation_width)
            img_chunck = clean_images(img_chunck)
            for j, (m, s) in enumerate(zip(means, stds)):
                img_chunck[:, :, :, j] -= m
                img_chunck[:, :, :, j] /= s
            
            img_new_shape = img_chunck.shape[1:]
            # escreve no arquivo novo
            if 'matrix' not in destf:
                destf.create_dataset('matrix', shape=(0,) + img_new_shape, maxshape=(None,) + img_new_shape)
            destf['matrix'].resize(destf['matrix'].shape[0] + img_chunck.shape[0], axis=0)
            destf['matrix'][-img_chunck.shape[0]:] = img_chunck

            info_chunck = src['info'][slc]
            if 'info' not in destf:
                destf.create_dataset('info', shape=(0,), maxshape=(None,))
            destf['info'].resize(destf['info'].shape[0] + info_chunck.shape[0], axis=0)
            destf['info'][-info_chunck.shape[0]:] = info_chunck


import os
pre_process(train_path, processed_train, img_w, mean, std, batch=load_batch)
os.remove(train_path)

pre_process(valid_path, processed_valid, img_w, mean, std, batch=load_batch)
os.remove(valid_path)

pre_process(test_path, processed_test, img_w, mean, std, batch=load_batch)
os.remove(test_path)


In [ ]:
def parse_example(image, label):
    image = tf.cast(image, tf.float32)
    image = preprocess_image_tf(image)
    return image, label

def preprocess_image_tf(image):
    angle_rad = tf.random.uniform([], 0, 2 * np.pi)
    image_shape = tf.shape(image)[0:2]
    cx = tf.cast(image_shape[1] / 2, tf.float32)
    cy = tf.cast(image_shape[0] / 2, tf.float32)
    cos_a = tf.math.cos(angle_rad)
    sin_a = tf.math.sin(angle_rad)
    transform = tf.stack([
        cos_a, -sin_a, (1 - cos_a) * cx + sin_a * cy,
        sin_a,  cos_a, (1 - cos_a) * cy - sin_a * cx,
        0.0,    0.0
    ])
    transform = tf.reshape(transform, [8])
    transform = tf.expand_dims(transform, 0)
    image = tf.expand_dims(image, 0)
    rotated = tf.raw_ops.ImageProjectiveTransformV3(
        images=image,
        transforms=transform,
        output_shape=image_shape,
        interpolation="BILINEAR",
        fill_mode="REFLECT",
        fill_value=0.0
    )
    rotated = tf.squeeze(rotated, 0)
    return tf.image.resize_with_crop_or_pad(rotated, img_w, img_w)

def load_dataset(file, batch_size):
    with h5py.File(file, 'r') as f:
        images = f['matrix'][:]
        labels = f['info'][:]

    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.repeat()
    dataset = dataset.shuffle(buffer_size=len(images))
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


In [ ]:
def get_data(file):
    with h5py.File(file, mode='r') as src:
        images = src['matrix'][:]
        info = src['info'][:]
        images = cut_images(images, img_w)
        return tf.constant(images), tf.constant(info)

In [ ]:
train_ds = load_dataset(processed_train, batch)
valid_ds = get_data(processed_valid)

In [ ]:

def build_model(input_shape, strides=(2, 2)):
    initializer = keras.initializers.RandomNormal(mean=0.0, stddev=0.01)
    reg = keras.regularizers.L2(1e-5)
    model = keras.models.Sequential()
    model.add(keras.layers.Input(input_shape))
    model.add(keras.layers.Conv2D(16, (4, 4), strides=strides, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    model.add(keras.layers.Conv2D(32, (3, 3), strides=strides, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    model.add(keras.layers.Conv2D(64, (3, 3), strides=strides, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    model.add(keras.layers.Conv2D(128, (3, 3), strides=strides, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    
    model.add(keras.layers.Flatten())
    
    model.add(keras.layers.Dense(256, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    model.add(keras.layers.Dense(64, activation='relu', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))
    model.add(keras.layers.Dense(1, activation='linear', kernel_initializer=initializer, kernel_regularizer=reg, bias_initializer=initializer, bias_regularizer=reg))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=.5e-4), loss='mse', metrics=['mse'])
    return model

In [ ]:
def training_number():
    counter = 0
    while True:
        yield counter
        counter += 1

training_n_gen = training_number()

In [ ]:
training_n = next(training_n_gen)

model = build_model((img_w, img_w, len(channels)),)
model.summary()

best_model_path = 'best_model.keras'
callback = tf.keras.callbacks.ModelCheckpoint(filepath=best_model_path,
                           monitor='val_loss',
                           verbose=0,
                           save_best_only=True,
                           mode='min')

with tf.device('/GPU:0'):
    history = model.fit(
        train_ds,
        validation_data=valid_ds,
        epochs=epochs,
        steps_per_epoch=iter_train,
        validation_steps=iter_valid,
        callbacks=[callback],
    )

model_info_dict = {
    "seed": seed,
    "shape": [
        img_w,
        img_w,
        len(channels)
    ],
    "channels": channels,
    "dataset": ds,
    "batch": batch,
     "normparams": [{"mean": mean[i], "std": std[i]} for i in range(len(channels))],
    "validmse": list(history['val_loss']),
    "trainingmse": list(history['loss'])
}

json_info = json.dumps(model_info_dict, indent=4)
with open(f'n{training_n}-model_info.json', 'w') as outfile:
    outfile.write(json_info)